In [1]:
%load_ext autoreload
%autoreload 2

In [79]:
import jax
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'
jax.devices()

[CpuDevice(id=0),
 CpuDevice(id=1),
 CpuDevice(id=2),
 CpuDevice(id=3),
 CpuDevice(id=4),
 CpuDevice(id=5),
 CpuDevice(id=6),
 CpuDevice(id=7)]

In [52]:
# import tensorboard
# %load_ext tensorboard
# %tensorboard --logdir logs
import tensorboard

AttributeError: module 'tensorboard' has no attribute 'SummaryWriter'

In [327]:
import flax
from symbiotic_transformers.cuisine_school import chef_config, chef, recipe_reader
path_to_text_file = "./recipes/recipes_small.txt"
config = chef_config.ChefConfig(n_times_to_imitate_chefs=2, n_training_recipes_per_imitation=100, n_exam_recipes_per_imitation=5)
brain_assistant = chef.BrainAssistant(config)
train_ds, exam_ds = recipe_reader.ImitateChefDataset(brain_assistant=brain_assistant).get_recipe_datasets(path_to_text_file)
student = chef.PromotedChef(config)

vocab size:  29


In [328]:
x, y = next(train_ds)
student.memorize_single_book(x, y)

iteration: 100 train loss: 0.18853533267974854 : 100%|██████████| 100/100 [00:59<00:00,  1.67it/s]


In [329]:
new_student = chef.PromotedChef(config)
new_student.load_chefs_brain()

vocab size:  29


In [330]:
yhat_decoded, yt_decoded = new_student.compare_sample(x, y)

In [331]:
print(yhat_decoded, yt_decoded)
# assert all(yhat_decoded == yt_decoded)

tf.Tensor(
[b'se casserole instructions preheat oven to  cook noodles drain brown meat and onion drain add tomatoes tomato sauce w'
 b'se mixture on top of crust mixture top with blueberry pie filling sprinkle top with remaining crust mixture refrigerate befor'
 b' minutes title creamed chicken over corn bread instructions melt butter add flour stir until mixed well add broth milk'
 b'from heat stir in sour cream and next  ingredients spoon into lightly greased quart baking dish sprinkle with breadcrumbs'], shape=(4,), dtype=string) tf.Tensor(
[b'se casserole instructions preheat oven to  cook noodles drain brown meat and onion drain add tomatoes tomato sauce w'
 b'se mixture on top of crust mixture top with blueberry pie filling sprinkle top with remaining crust mixture refrigerate befor'
 b' minutes title creamed chicken over corn bread instructions melt butter add flour stir until mixed well add broth milk'
 b'from heat stir in sour cream and next  ingredients spoon into lightly gre

In [343]:
new_student.generate_new_recipe("creamed chicken")

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'title creamed chicken instructions nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddndddddndndddddnddddnddnc    p   n'],
      dtype=object)>

In [342]:
import tensorflow as tf
import jax.numpy as jnp
title="creamed chicken over corn bread instructions"
template = f"title: {title} instructions: "

state = flax.jax_utils.unreplicate(new_student.chef_brain)
tokens = new_student.brain_assistant.transform_recipe_for_training([template])
tokens = jnp.array(tokens.numpy())
tokens = jnp.reshape(jnp.pad(tokens[0], (new_student.config.max_seq_len + 1 - tokens.shape[1], 0), 'constant', constant_values=0), (1, -1))
for _ in range(100):
    # pylint: disable=not-callable
    character_logits = state.apply_fn(
        {"params": state.params},
        tokens[:, -new_student.config.max_seq_len :],
        rngs={"dropout": jax.random.PRNGKey(0)},
        training=False,
    )
    next_character_logits = character_logits[:, -1:, :]
    tokens = jnp.concatenate(
        [tokens, jnp.argmax(next_character_logits, axis=-1)], axis=-1
    )
tf.strings.reduce_join( new_student.brain_assistant.itos(tokens), axis=1) 
# new_student.brain_assistant.decode_chefs_articulated_idea(tokens)
# # pad array to max_seq_len
# print(tokens.shape)


# # character_logits = state.apply_fn(
# #     {"params": state.params},
# #     tokens,
# #     rngs={"dropout": jax.random.PRNGKey(0)},
# #     training=False,
# # )

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'[UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK]title creamed chicken over corn bread instructions instructions nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnndnnnnnnnn n    ep  p     n    n       d dd      dc'],
      dtype=object)>

In [305]:
tokens = new_student.brain_assistant.transform_recipe_for_training([template])
tokens = jnp.array(tokens.numpy())
print(tokens.shape)
jnp.pad(tokens[0], (0, new_student.config.max_seq_len + 1 - tokens.shape[1]), 'constant', constant_values=0)

(1, 35)


Array([ 3,  5,  3, 11,  2,  4,  1,  5,  3,  7, 11,  5,  7,  6,  1, 16,  7,
       10,  3,  7,  1,  5,  6, 10,  3,  9, 14, 13,  3,  5,  8,  6, 10,  4,
        1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [300]:
tokens.shape

(95, 129)

In [268]:
student.become_a_master_chef(train_ds, exam_ds)

  0%|          | 0/2 [00:00<?, ?it/s]

[3.3619316 3.3825345 3.3777065 3.3977609 3.3663912 3.3970602 3.3784795
 3.3928924]
3.3818445


replication: 0 train loss: 3.3818445205688477 eval loss: 3.3566274642944336:  50%|█████     | 1/2 [00:04<00:04,  4.56s/it]

{'train': [3.3818445205688477], 'eval': [3.3566274642944336]}


replication: 1 train loss: 3.360645294189453 eval loss: 3.353447914123535: 100%|██████████| 2/2 [00:06<00:00,  3.11s/it]  

[3.3619926 3.3738596 3.3442163 3.3286252 3.3801181 3.3608825 3.3759947
 3.3594744]
3.3606453
{'train': [3.360645294189453], 'eval': [3.353447914123535]}


replication: 1 train loss: 3.360645294189453 eval loss: 3.353447914123535: 100%|██████████| 2/2 [00:07<00:00,  3.63s/it]


In [280]:
import flax
from symbiotic_transformers.cuisine_school import chef_config, chef, recipe_reader
path_to_text_file = "./recipes/recipes_small.txt"
config = chef_config.ChefConfig(n_times_to_imitate_chefs=2, n_training_recipes_per_imitation=5, n_exam_recipes_per_imitation=5)
brain_assistant = chef.BrainAssistant(config)
train_ds, exam_ds = recipe_reader.ImitateChefDataset(brain_assistant=brain_assistant).get_recipe_datasets(path_to_text_file)
student = chef.Chef(config)

vocab size:  29


In [276]:
x, y = next(train_ds)

In [279]:
x.shape, y.shape

((32, 128), (32, 128, 29))

In [277]:
prng = jax.random.PRNGKey(0)
chef.train_step_jitted(student.chef_brain, x, y, prng)

(TrainState(step=Array(1, dtype=int32, weak_type=True), apply_fn=<bound method Module.apply of ChefBrain(
     # attributes
     max_seq_len = 128
     brain_size = 256
     n_ideas = 2
     n_moldings = 2
     dropout_rate = 0.2
     chef_vocabulary_size = 29
 )>, params={'IdeaArticulation_0': {'Dense_0': {'bias': Array([-9.9998455e-05, -9.9998004e-05, -9.9996541e-05, -9.9997407e-05,
         0.0000000e+00, -9.9998295e-05, -9.9992030e-05,  9.9991281e-05,
         0.0000000e+00,  0.0000000e+00, -9.9997771e-05, -9.9995101e-05,
        -9.9991928e-05, -9.9998375e-05, -9.9998557e-05, -9.9998171e-05,
        -9.9995035e-05,  0.0000000e+00,  0.0000000e+00,  9.9989280e-05,
        -9.9997953e-05, -9.9987439e-05, -9.9998833e-05, -9.9998106e-05,
        -9.9997298e-05, -9.9992656e-05, -9.9997960e-05,  9.9964178e-05,
        -9.9995348e-05, -9.9998702e-05,  0.0000000e+00, -9.9991950e-05,
         0.0000000e+00,  9.9995901e-05, -9.9984238e-05,  9.9977544e-05,
        -9.9844037e-05, -9.9997480e-

In [94]:
f"Predicted: {yhat}"

"Predicted: b'xfgzalgojmqbnwswlmugtsyrqrhjdnfx e cpu evgkdnuhqgkm xxn[UNK]qjyovvijfrfackkvfnlxehhzphwduh ls[UNK]frwfjcg gaidwzrxouzz[UNK]sbzihlayrq anzs'"

In [95]:

# show an example of writing to tensorboard
import tensorflow as tf
logdir = "./logs"
file_writer = tf.summary.create_file_writer(logdir)
# write 

def write_to_tensorboard(epoch, loss, logits, y):
    with file_writer.as_default():
        with tf.name_scope("train"):
            tf.summary.scalar('loss', loss, step=epoch)

            with tf.name_scope("samples"):
                tf.summary.text("predicted", f"""
                                MasterChef: {student.brain_assistant.decode_chefs_articulated_idea(logits)[0]}
                                StudenChef: {student.brain_assistant.decode_chefs_articulated_idea(y)[0]}
                                """
                                , step=0)
                tf.summary.text("target", student.brain_assistant.decode_chefs_articulated_idea(y)[0] , step=0)


        

for epoch in range(10):
    for step in range(10):
        prng, _ = jax.random.split(prng)
        student.chef_brain, (loss, logits) = chef.train_step_jitted(student.chef_brain, x, y, prng)
        print(loss)

    for step in range(10):
        prng, _ = jax.random.split(prng)
        (loss, logits) = chef.eval_step_jitted(student.chef_brain, x, y, prng)
        print(loss)
        
    with file_writer.as_default():
        with tf.name_scope("train"):
            tf.summary.scalar('loss', loss, step=epoch)

            with tf.name_scope("samples"):
                tf.summary.text("predicted", f"""
                                MasterChef: {student.brain_assistant.decode_chefs_articulated_idea(logits)[0]}
                                StudenChef: {student.brain_assistant.decode_chefs_articulated_idea(y)[0]}
                                """
                                , step=0)
                tf.summary.text("target", student.brain_assistant.decode_chefs_articulated_idea(y)[0] , step=0)



3.435975
3.3537064
3.340958
3.336102
3.3191836
3.3133993
3.2958927
3.279913
3.2584925
3.2453272
3.1822934
3.1822934
3.1822934
3.1822934
3.1822934
3.1822934
3.1822934
3.1822934
3.1822934
3.1822934
3.223957
3.201137
3.177502
3.1587677
3.1251163
3.1171093
3.0820062
3.0501313
3.0336747
2.9950566
2.86533
2.86533
2.86533
2.86533
2.86533
2.86533
2.86533
2.86533
2.86533
2.86533
2.9685583
2.9307013
2.8974972
2.8852134
2.844458
2.8062508
2.778759
2.7301824
2.7166972
2.6831722
2.469361
2.469361
2.469361
2.469361
2.469361
2.469361
2.469361
2.469361
2.469361
2.469361
2.63906
2.6012583
2.5689473
2.5386894
2.4871743
2.455368
2.4113727
2.3797216
2.317053
2.3086643
2.0209804
2.0209804
2.0209804
2.0209804
2.0209804
2.0209804
2.0209804
2.0209804
2.0209804
2.0209804
2.2738557
2.2161124
2.179731
2.1474786
2.1067526
2.046114
2.0192504
1.9505365
1.9126439
1.884164
1.5023766
1.5023766
1.5023766
1.5023766
1.5023766
1.5023766
1.5023766
1.5023766
1.5023766
1.5023766
1.8266107
1.7778497
1.6981409
1.6690128
1.6227

  0%|          | 0/4 [00:00<?, ?it/s]


ScopeParamShapeError: Initializer expected to generate shape (8, 29, 256) but got shape (29, 256) instead for parameter "embedding" in "/StructureInformation_0/Embed_0". (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.ScopeParamShapeError)

In [19]:
ptrain_step = jax.pmap(chef.parallel_train_step_jitted, axis_name='batch')
peval_step = jax.pmap(chef.parallel_eval_step_jitted, axis_name='batch')

state = flax.jax_utils.replicate(student.chef_brain)

In [20]:
next(train_ds)

(array([[ 1,  4,  4, ..., 14, 13,  3],
        [ 8,  6,  5, ...,  1, 16,  7],
        [13,  2,  6, ...,  7,  3,  1],
        ...,
        [ 7, 24,  2, ..., 10,  1,  5],
        [ 5,  9,  1, ...,  1, 13,  8],
        [13,  2,  1, ...,  8, 17, 19]]),
 array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 0., .

In [21]:
import tensorflow as tf


In [22]:
jax.local_device_count()

8

In [23]:
from tqdm import tqdm
from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
rng = jax.random.PRNGKey(config.kitchen_seed)
dropout_rngs = jax.random.split(rng, jax.local_device_count())
n_epochs = 2
n_steps_per_epoch = 128
for i, epoch in enumerate(tqdm(range(1, n_epochs + 1), desc=f"Epoch ...", position=0, leave=True)):
    rng, input_rng = jax.random.split(rng)

    # train
    with tqdm(total=n_steps_per_epoch // config.batch_size, desc="Training...", leave=False) as progress_bar_train:
        for _ in range(10):
            x, y = next(train_ds)
            x, y = shard(x), shard(y)
            state, (loss, logits, train_metrics) = ptrain_step(state, x, y, dropout_rngs)
            dropout_rngs = jax.random.split(dropout_rngs[0], jax.local_device_count())
            progress_bar_train.update(1)



Epoch ...:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch ...: 100%|██████████| 2/2 [00:14<00:00,  7.06s/it]


In [54]:
loss

Array([3.0181525, 3.0156236, 3.041226 , 3.0524318, 2.975197 , 3.0833395,
       2.9946923, 2.997703 ], dtype=float32)

In [42]:
round(flax.jax_utils.unreplicate(train_metrics)['loss'], 3)

Array(3.1060002, dtype=float32)

In [51]:
peval_step(state, x, y, dropout_rngs)

TypeError: 'TrainState' object is not subscriptable

In [32]:

out = peval_step(state, shard(x), shard(y), dropout_rngs)

In [35]:
import jax.numpy as jnp
loss, logits = out
jnp.mean(loss)

Array(3.4194553, dtype=float32)

In [2]:
import tensorflow as tf

In [3]:
import tensorflow as tf

In [186]:
import tqdm
path_to_text_file = "./recipes/recipes_small.txt"
config = chef_config.ChefConfig()
brain_assistant = chef_gpt_brain.BrainAssistant(config)
train_ds, exam_ds = recipe_reader.ImitateChefDataset(brain_assistant=brain_assistant).get_recipe_datasets(path_to_text_file)
chef = chef_gpt_brain.Chef(config)

# for _ in range(3):
#     for i, (x, y) in tqdm.tqdm(enumerate(train_ds)):
#         if i > 10:
#             break
#         print(x.shape, y.shape)

for x, y in train_ds:
    print(x.shape, y.shape)
    break

# for x, y in exam_ds:
#     print(x.shape, y.shape)
#     break





vocab size:  29
(32, 128) (32, 128, 29)


In [187]:
chef.become_a_master_chef(train_ds, exam_ds)

5001it [37:36,  2.22it/s]
101it [00:07, 14.16it/s]


replication: 0 train loss: 2.6830525398254395 eval loss: 2.1154134273529053

                recipe as written by master:
                [b'kberry cobbler instructions mix together flour baking powder and salt cream together  cup sugar eggs butter and vanilla a'
 b'n under broiler title salami instructions mix and wrap in saran wrap refrigerate  hours remove wrap form into  rolls '
 b'and  cool whip repeat steps and top with chopped nuts title raspberry pretzel dessert instructions mix together press i']
                recipe as written by chef:
                [b'  rise sa eaed in te tione med t o t e tinte citane reane  aod iotee aaes trtetien  aer en ae eote title aed  tiree  '
 b' to e s ine  ctitee sotel instructions cox and cnoteen soaer inaee oee s  aoe   toe e  toed  oat air  oitae srtes  '
 b' e id tatn  oete adeare ean  and aod tatn coo aed iotr  to ee uttine cott e  e tteat instructioee min tructrons  ent a']
                


5001it [22:57,  3.63it/s]
101it [00:06, 15.07it/s]


replication: 1 train loss: 2.2786715030670166 eval loss: 1.9918795824050903

                recipe as written by master:
                [b'as directed on box when tender drain on low heat add milk butter and american cheese when cheese is melted remove from hea'
 b'ctions trim off the green and pink parts of the rind and cut into cubes when you have   quarts prepared cover with water '
 b'egg yolks together place in a large bowl in layers the cake filling and butterfingers title smoked oyster ball instructions']
                recipe as written by chef:
                [b' t rate iet in cent saer  inee aas  in  in teg reat  and mile  atten and entt aed coetl  toel  oease ie tit er he ten  iet r  '
 b' tions tots red teeee aes and bane  ate  ae iee han  and re ter  mitee toen and   an  toe  itee  ur ines coee ait  hiter '
 b'ne cote aid tran crale an  ritae tiee oe te  tee  ale min ies and rutte it sns citles saaae  an te tiue in  r ceioe ']
                


5001it [23:02,  3.62it/s]
101it [00:06, 15.13it/s]


...

5001it [22:51,  3.65it/s]
101it [00:06, 15.56it/s]


replication: 41 train loss: 1.5226142406463623 eval loss: 1.0121186971664429

                recipe as written by master:
                [b'eese when cheese is melted remove from heat and serve mozzarella is optional title caramel fudge cake squares instructions'
 b'r colored add lemon juice place broccoli in serving dish pour lemon butter over top title jello cubes for kids instruction'
 b'r bake in  oven until browned on top title tillie scotch bars instructions cream margarine then gradually add both su']
                recipe as written by chef:
                [b'eeler shen shrase an retle  remare  hor reat and serle mattesell tis  r ion r tiole cotetee sitse cake s  ares instructions'
 b'd cokeles  and reton coite poare are celewin serven  meth tour le in bitter aner tot title cotle cilosetor reds instruction'
 b'e boke in  over untir breanes intter title torl e shouct cens instructions creanemurgerine tlen glal age  add bathes ']
                


5001it [26:14,  3.18it/s]
101it [00:09, 10.71it/s]


replication: 42 train loss: 1.5198686122894287 eval loss: 1.0072829723358154

                recipe as written by master:
                [b' brown bacon in skillet for  minutes put bacon on paper towels pour off fat use  tablespoons fat in skillet put chopped o'
 b' or until melted blend garlic salt paprika and cheese quarter potatoes lengthwise dip cut edges in butter mixture arrange c'
 b'ble bacon quiche instructions preheat oven to  lightly grease a inch pie plate sprinkle bacon cheese and onion in pie']
                recipe as written by chef:
                [b' criln cocon an srille pore minutes pit bocon on saper cekoes poer ont oas ate  totlestoont pur in srilnes pit shopled  '
 b' or intir ca les blen sarlin sand torruce and chlale   unter wnt raed len telite dit clt entes in bitter mixt r mar inge s'
 b'toe bater s icke instructions poereat oven to  tinhtle arease ainth pie tlele sp intle bican solase and inion in tun']
                


187it [01:14,  1.07s/it]

In [185]:
import jax
import jax.numpy as jnp
print(tf.strings.reduce_join(chef.brain_assistant.itos(x), axis=1)[:5])
character_logits = chef.chef_brain.apply_fn(
                {"params": chef.chef_brain.params},
                jnp.asarray(x),
                rngs={"dropout": jax.random.PRNGKey(0)},
                training=False,
            )
print(chef.brain_assistant.decode_chefs_articulated_idea(character_logits)[:5])
print(chef.brain_assistant.decode_chefs_articulated_idea(y)[:5])

tf.Tensor(
[b'g  teaspoons drop and shape into  clusters on wax paper let stand until firm about  minutes title jewell balls chick'
 b'al mix well using  teaspoons drop and shape into  clusters on wax paper let stand until firm about  minutes title j'
 b'stir  minutes more take off heat stir in vanilla and cereal mix well using  teaspoons drop and shape into  clusters on '
 b'gar nuts evaporated milk and butter or margarine stir over medium heat until mixture bubbles all over top boil and stir  mi'
 b'ookies instructions in a heavy quart saucepan mix brown sugar nuts evaporated milk and butter or margarine stir over medi'], shape=(5,), dtype=string)
tf.Tensor(
[b'  teaspoons drop and shape into  clusters on wax paper let stand until firm about  minutes title jewell balls chicke'
 b'l mix well using  teaspoons drop and shape into  clusters on wax paper let stand until firm about  minutes title je'
 b'tir  minutes more take off heat stir in vanilla and cereal mix well using  teaspoons dr

In [142]:
new_chef = chef_gpt_brain.Chef(config)
new_chef.load_chefs_brain()

vocab size:  29


ValueError: NOT_FOUND: Error opening "zarr" driver: Metadata at "config.chef_state_path/.zarray" in OCDBT database at local file "/tmp/ckpt/orbax/managed/1/default/" does not exist [tensorstore_spec='{\"context\":{\"cache_pool\":{},\"cache_pool#ocdbt\":{\"total_bytes_limit\":100000000},\"data_copy_concurrency\":{},\"file_io_concurrency\":{\"limit\":128},\"file_io_sync\":true,\"ocdbt_coordinator\":{}},\"driver\":\"zarr\",\"kvstore\":{\"base\":{\"driver\":\"file\",\"path\":\"/tmp/ckpt/orbax/managed/1/default/\"},\"cache_pool\":\"cache_pool#ocdbt\",\"driver\":\"ocdbt\",\"experimental_read_coalescing_interval\":\"1ms\",\"experimental_read_coalescing_merged_bytes\":500000000000,\"experimental_read_coalescing_threshold_bytes\":1000000,\"path\":\"config.chef_state_path/\"},\"recheck_cached_data\":false,\"recheck_cached_metadata\":false}'] [source locations='tensorstore/driver/kvs_backed_chunk_driver.cc:1256\ntensorstore/driver/driver.cc:109']

In [78]:
chef.become_a_master_chef(train_ds, exam_ds)

TypeError: mean requires ndarray or scalar arguments, got <class 'list'> at position 0.

In [14]:
config = chef_config.ChefConfig()
path_to_text_file = "./recipes/recipes_small.txt"
# text = open(path_to_text_file, "rb").readlines().decode(encoding="utf-8")[:config.all_recipes_size]

with open(path_to_text_file, "r") as f:
    text = " ".join(f.readlines())[:config.all_recipes_size]
    text = text[:config.all_recipes_size]
text

"title: No-Bake Nut Cookies instructions: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.\n title: Jewell Ball'S Chicken instructions: Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.\n title: Creamy Corn instructions: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.\n title: Chicken Funny instructions: Boil and debone chicken. Put bite size pieces in average size square casserole dish. Pour gravy and cream of mushroom soup over chicken; level. Make stuffing according to in

In [36]:

class ImitateChefDataset:
    def __init__(self, chef: chef_gpt_brain.Chef):
        self.chef = chef

    def to_batch(self, x, window_size, batch_size):
        max_value = tf.cast(tf.shape(x)[0] - window_size, tf.int64)
        idxs = tf.experimental.numpy.random.randint(0, max_value, size=batch_size)

        # some weird tf stuff...
        def slice_sequence(idx):
            return x[idx : idx + window_size]

        batch = tf.map_fn(slice_sequence, idxs, dtype=tf.int64)
        return batch

    def _build_recipe_ds(self, text: str, is_train):

        imitation_ds = tf.data.Dataset.from_tensor_slices([text])

        if is_train:
            # fit
            fit_ds = imitation_ds.map(self.chef._clean_text_and_convert_to_characters)
            self.chef._build_chef_vocabulary(fit_ds)
        # transform
        imitation_ds = imitation_ds.map(self.chef.transform_recipe_for_training)
        imitation_ds = imitation_ds.map(
            lambda x: self.to_batch(
                x,
                self.chef.config.max_seq_len + 1,
                self.chef.config.num_subordinates_per_shop * self.chef.config.n_shops,
            )
        )
        imitation_ds = imitation_ds.map(lambda x: (x[:, :-1], x[:, 1:]))

        # convert to one hot labels
        imitation_ds = imitation_ds.map(
            lambda x, y: (x, tf.one_hot(y, int(self.chef.stoi.vocabulary_size())))
        )

        return imitation_ds.repeat().prefetch(tf.data.AUTOTUNE).as_numpy_iterator()

    def get_recipe_datasets(self, path_to_text_file: str):
        text = (open(path_to_text_file, "rb").read().decode(encoding="utf-8"))[
            :self.chef.config.all_recipes_size
        ]
        train_text, eval_text = (
            text[: int(len(text) * 0.9)],
            text[int(len(text) * 0.9) :],
        )

        train_ds = self._build_recipe_ds(train_text, is_train=True)
        eval_ds = self._build_recipe_ds(eval_text, is_train=False)
        return train_ds, eval_ds

In [41]:
config = chef_config.ChefConfig()
chef = chef_gpt_brain.Chef(config)
imitator_ds = ImitateChefDataset(chef=chef)
train_ds, exam_ds = imitator_ds.get_recipe_datasets(path_to_text_file)
for x, y in train_ds:
    print(x, y)
    break

print(chef.stoi.vocab_size(), chef.stoi.get_vocabulary())

chef.decode_chefs_articulated_idea(y)
# print(tf.strings.reduce_join(chef.itos(x)))
# tf.strings.reduce_join(chef.itos(x), axis=1)


[[ 3  9  2 ...  4  6 12]
 [ 3  9  2 ...  3  2 18]
 [ 5 23  7 ... 11  4  6]
 ...
 [ 2 13 10 ... 21 21  3]
 [11  4  8 ...  2  8  7]
 [12 12  2 ...  2  4  5]] [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0.

30 ['|', '?', ' ', 'e', 't', 'o', 'i', 'n', 'a', '', 'r', 's', 'l', 'c', 'd', 'u', 'p', 'h', 'm', 'b', 'g', 'f', 'k', 'w', 'v', 'y', 'x', 'q', 'z', 'j']


<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'  tablespoons of green onion and  cup of bacon cook until heated do not boil stir in sour cream if desired cook until ',
       b' cut in half lengthwise scoop out pulp and reserve discard shells brown ground beef until done drain any grease from the me',
       b'wn sugar nuts evaporated milk and butter or margarine stir over medium heat until mixture bubbles all over top boil and stir',
       b'n top of beef mix soup and cream together pour over chicken bake uncovered at  for  hourstitle creamy corn instruct',
       b'ese on top and bake at  for approximately  minutes or until golden and bubblytitle reeses cupscandy   instructions c',
       b' using a toothpick dip balls  of way into chocolate chip and paraffin mixture to make them look like buckeyestitle quick ',
       b' boil and stir  minutes more take off heat stir in vanilla and cereal mix well using  teaspoons drop and shape into  c',
       b' fork microwave them with 

In [24]:
chef.stoi.get_vocabulary()

['|',
 '?',
 ' ',
 'e',
 't',
 'i',
 'r',
 'a',
 '',
 's',
 'n',
 'o',
 'c',
 'p',
 'u',
 'l',
 'm',
 'd',
 'h',
 'f',
 'w',
 'x',
 'g',
 'z',
 'v',
 'b',
 'y',
 'k']

In [15]:
x

In [4]:
import jax
import jax.numpy as jnp
import flax
import flax.linen as nn
import chex


class PositionalEncoding(nn.Module):
    # positional encoding enables the model to know the position of the token in the sequence
    # we do this by adding a positional encoding vector to the feature vector.
    # normally, we would add it to an embedding vector, but since we working with time series data,
    # we don't have an embedding vector, so we add it to the feature vector.

    max_seq_len: int
    brain_size: int

    def setup(self):
        # first, we create a matrix of shape (max_seq_len, brain_size)
        pos_enc = jnp.zeros((self.max_seq_len, self.brain_size))

        raw_position = jnp.arange(0, self.max_seq_len)[:, jnp.newaxis]
        chex.assert_shape(raw_position, (self.max_seq_len, 1))
        # pos / (10000 ** (2 * i / brain_size))
        divider = 10000 ** (2 * jnp.arange(0, self.brain_size, 2) / self.brain_size)

        sin_encoding = jnp.sin(raw_position / divider)
        cos_encoding = jnp.cos(raw_position / divider)

        # overwrite the feature dimension.
        pos_enc = pos_enc.at[:, 0::2].set(sin_encoding)
        pos_enc = pos_enc.at[:, 1::2].set(cos_encoding)

        # add batch dimension
        self.pos_enc = pos_enc[jnp.newaxis, :, :]

    def __call__(self, x, add_to_inputs=True):
        # select all elements in batch, up to timeseries length, and all features
        if add_to_inputs:
            return x + self.pos_enc[:, :x.shape[1], :]
        return self.pos_enc[:, :x.shape[1], :]


class StructureInformation(nn.Module):
    max_seq_len: int
    brain_size: int
    chef_vocab_size: int

    @nn.compact
    def __call__(self, x, training=True):
        # x: (batch_size, seq_len)
        # we want to add a layer norm to the features dimension
        B, T = jnp.shape(x)
        
        token_embeddings = nn.Embed(self.chef_vocab_size, self.brain_size)(x) # (batch_size, seq_len, brain_size)
        position_embeddings = nn.Embed(self.max_seq_len, self.brain_size)(jnp.arange(0, T)) 

        x = token_embeddings + position_embeddings
        
        # x = nn.LayerNorm()(x)
        # # we want to add a dropout layer
        # x = nn.Dropout(0.1)(x, deterministic=not training)

        # # convert to (batch_size, seq_len, brain_size)
        # x = nn.Dense(features=self.brain_size)(x)

        # # add positional encoding
        # x = PositionalEncoding(self.max_seq_len, self.brain_size)(x)
        return x




# Could have been a function, but we want to use the @nn module
class Thought(nn.Module):
    # Scaled dot product attention
    brain_size: int

    def setup(self):
        self.questions_to_improve_knowledge = nn.Dense(features=self.brain_size) # queries
        self.knowledge_index = nn.Dense(features=self.brain_size) # keys
        self.information_in_knowledge_index = nn.Dense(features=self.brain_size) # values


    @nn.compact
    def __call__(self, x): #given_a_question, knowledge_index, information_in_knowledge_index):
        B, T, C = jnp.shape(x)

        formulated_question = self.questions_to_improve_knowledge(x)
        knowledge_index = self.knowledge_index(x)
        information_in_indices = self.information_in_knowledge_index(x)

        attention_to_relevant_indexes = jnp.matmul(formulated_question, jnp.swapaxes(knowledge_index, -2, -1)) / jnp.sqrt(self.brain_size)
        # mask out the future tokens
        mask = jnp.tril(jnp.ones((T, T))) # tril = lower triangle
        attention_to_relevant_indexes = jnp.where(mask == 0, -jnp.inf, attention_to_relevant_indexes)

        
        probability_knowledge_index_is_relevant = nn.softmax(attention_to_relevant_indexes, axis=-1)
        answer_to_question_based_on_information_in_knowledge_index = jnp.matmul(probability_knowledge_index_is_relevant, information_in_indices)
        # return both for debugging purposes
        return answer_to_question_based_on_information_in_knowledge_index, attention_to_relevant_indexes


In [5]:
class BrainStorm(nn.Module):
    n_ideas: int
    brain_size: int

    def setup(self) -> None:
        chex.assert_equal(self.brain_size % self.n_ideas, 0)
        self.idea_size = self.brain_size // self.n_ideas

        self.thoughts = [Thought(self.idea_size) for _ in range(self.n_ideas)]
        
        self.filter_interesting_thoughts = nn.Dense(
            features=self.brain_size,
            kernel_init=nn.initializers.xavier_uniform(),
            use_bias=False,
        )

    def __call__(self, x):
        thoughs = jnp.concatenate([thought(x)[0] for thought in self.thoughts], axis=-1)
        interesting_thoughts = self.filter_interesting_thoughts(thoughs)
        return interesting_thoughts


In [6]:

class ProjectIdeas(nn.Module):
    brain_size: int
    dropout_rate: float

    @nn.compact
    def __call__(self, x, training=True):
        x = nn.Dense(
            features=4 * self.brain_size,
            kernel_init=nn.initializers.xavier_uniform(),
            use_bias=False,
        )(x)
        x = nn.relu(x)
        x = nn.Dense(
            features=self.brain_size,
            kernel_init=nn.initializers.xavier_uniform(),
            use_bias=False,
        )(x)
        x = nn.Dropout(self.dropout_rate)(x, deterministic=not training)
        return x


class CreativityBlock(nn.Module):
    brain_size: int
    n_ideas: int
    dropout_rate: float

    def setup(self):
        # ln -> mha -> projection -> ln
        # Andrej Karpathy min 1:35:30 https://www.youtube.com/watch?v=kCc8FmEb1nY --> move layernorm before mha
        self.normalize_content1 = nn.LayerNorm()
        self.brainstrom = BrainStorm(self.n_ideas, self.brain_size)
        self.normalize_content2 = nn.LayerNorm()
        self.project_ideas = ProjectIdeas(self.brain_size, self.dropout_rate)
    
    def __call__(self, x, training=True):
        # lha -> brainstrom
        x = self.normalize_content1(x)
        x = self.brainstrom(x)

        # residual connection
        x = x + x

        # ln -> projection
        x = self.normalize_content2(x)
        x = self.project_ideas(x, training=training)

        # residual connection
        x = x + x
        return x

class IdeaIteration(nn.Module):
    n_moldings: int
    brain_size: int
    n_ideas: int
    dropout_rate: float

    def setup(self):
        self.creativity_blocks = [
            CreativityBlock(self.brain_size, self.n_ideas, self.dropout_rate)
            for _ in range(self.n_moldings)
        ]
    
    def __call__(self, x, training=True):
        for creativity_block in self.creativity_blocks:
            x = creativity_block(x, training=training)
        return x
    

class IdeaArticulation(nn.Module):
    max_seq_len: int
    brain_size: int
    dropout_rate: float
    chef_vocabulary_size: int

    @nn.compact
    def __call__(self, x, training=True):
        # flatten
        x = jnp.reshape(x, (x.shape[0], -1)) 
        x = nn.LayerNorm()(x)
        x = nn.Dense(features=self.brain_size, kernel_init=nn.initializers.xavier_uniform())(x)
        x = nn.relu(x)
        x = nn.Dropout(self.dropout_rate)(x, deterministic=not training)
        x = nn.Dense(features=self.max_seq_len*self.chef_vocabulary_size, kernel_init=nn.initializers.xavier_uniform())(x)
        x = jnp.reshape(x, (x.shape[0], self.max_seq_len, self.chef_vocabulary_size))
        return x

In [7]:
class ChefBrain(nn.Module):
    max_seq_len: int
    brain_size: int
    n_ideas: int
    n_moldings: int
    dropout_rate: float
    chef_vocabulary_size: int

    @nn.compact
    def __call__(self, x, training=True):
        x = StructureInformation(self.max_seq_len, self.brain_size, self.chef_vocabulary_size)(x, training=training)
        x  = IdeaIteration(self.n_moldings, self.brain_size, self.n_ideas, self.dropout_rate)(x, training=training)
        x = IdeaArticulation(self.max_seq_len, self.brain_size, self.dropout_rate, self.chef_vocabulary_size)(x, training=training)
        return x

In [8]:
import optax
from flax.training import train_state

def prepare_chef_for_training(chef_config: chef_config.ChefConfig):
    rng = jax.random.PRNGKey(chef_config.kitchen_seed)

    model_rng, dropout_rng = jax.random.split(rng)
    # learning_rate = 3e-4
    # tx = optax.adam(learning_rate=learning_rate)

    # warmup_cosine_decay_scheduler
    total_steps = chef_config.n_times_to_imitate_chefs * chef_config.n_recipes_to_sample
    learn_by_scanning_followed_by_deepdive = optax.warmup_cosine_decay_schedule(init_value=0.0001, peak_value=0.001,
                                                                   warmup_steps=int(total_steps*0.2),
                                                                   decay_steps=total_steps, end_value=0.00001)
    tx = optax.adam(learn_by_scanning_followed_by_deepdive)

    model = ChefBrain(
        max_seq_len=chef_config.max_seq_len,
        brain_size=chef_config.brain_size,
        n_ideas=chef_config.n_ideas,
        n_moldings=chef_config.n_moldings,
        dropout_rate=chef_config.dropout_rate,
        chef_vocabulary_size=chef_config.chef_vocab_size
    )

    variables = model.init(
        {
            'params': model_rng,
            'dropout': dropout_rng
        },
        jnp.ones((chef_config.batch_size, chef_config.max_seq_len), dtype=jnp.int32)
    )
    
    state = train_state.TrainState.create(
            apply_fn = model.apply,
            tx=tx,
            params=variables['params']
        )
    return state

In [9]:
def train_step(state, x, y, dropout_rng=None):
    dropout_rng = jax.random.fold_in(dropout_rng, state.step)
    def loss_fn(params):
        logits = state.apply_fn({"params": params}, x, rngs={"dropout": dropout_rng})
        loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    return state, (loss, logits)

@jax.jit
def train_step_jitted(state, x, y, dropout_rng=None):
    dropout_rng = jax.random.fold_in(dropout_rng, state.step)
    def loss_fn(params):
        logits = state.apply_fn({"params": params}, x, rngs={"dropout": dropout_rng})
        loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    return state, (loss, logits)

    
@jax.jit
def eval_step_jitted(state, x, y, dropout_rng=None):
    logits = state.apply_fn({"params": state.params}, x, rngs={"dropout": dropout_rng}, training=False)
    loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
    return loss, logits


In [15]:
from orbax import checkpoint

def train_and_eval(chef, train_ds, test_ds, chef_config: chef_config.ChefConfig):
    rng = jax.random.PRNGKey(chef_config.kitchen_seed)
    orbax_checkpointer = checkpoint.PyTreeCheckpointer()
    options = checkpoint.CheckpointManagerOptions(max_to_keep=2, create=True)
    checkpoint_manager = checkpoint.CheckpointManager(chef_config.chef_state_path, orbax_checkpointer, options)

    ckpt = {"model": chef}
    for replication in range(chef_config.n_times_to_imitate_chefs):
        for batch_idx, (recipe_instructions, recipe_by_master) in enumerate(train_ds.take(chef_config.n_recipes_to_sample).as_numpy_iterator()):
            rng, dropout_rng = jax.random.split(rng)
            recipe_by_master = jax.nn.one_hot(recipe_by_master, chef_config.chef_vocab_size)
            chef, (loss, logits) = train_step_jitted(chef, recipe_instructions, recipe_by_master, dropout_rng)

            if batch_idx % 100 == 0:
                # print relevant information
                print(recipe_reader.decode(x, itos)[:5])
                print(recipe_reader.decode(jnp.argmax(jnp.squeeze(logits), axis=-1), itos)[:5])
                print(f"Replication: {replication}, batch: {batch_idx}, loss: {loss}")
    
        for batch_idx, (recipe_instructions, recipe_by_master) in enumerate(test_ds.take(chef_config.n_exam_recipes).as_numpy_iterator()):
            rng, dropout_rng = jax.random.split(rng)
            recipe_by_master = jax.nn.one_hot(recipe_by_master, chef_config.chef_vocab_size)
            loss, logits = eval_step_jitted(chef, recipe_instructions, recipe_by_master, dropout_rng)

        print(f"Evaluation --- replication: {replication}, batch: {batch_idx}, loss: {loss}")
        checkpoint_manager.save(replication, ckpt)
    return chef

In [16]:
config = chef_config.ChefConfig(chef_vocab_size=len(stoi.get_vocabulary()))
chef = prepare_chef_for_training(config)
trained_chef = train_and_eval(chef, imitation_ds_training, imitation_ds_exam, config)

tf.Tensor(
[b'ctions prepare peppers remove white sections mix ground beef rice egg and seasoning fill peppers place in deep cooking v'
 b'htly salted boiling water cook red cabbage for  minutes then draintitle german potato salad instructions peel and cut pot'
 b'gs bring to a boil and then reduce heat cover and simmer for  to  hours during the last hour of cooking add the winetitl'
 b'soup sour cream and cheese stir well add potatoes to mixture and pour into a  quart casserole dish crush cornflakes mi'
 b'yield   dozentitle punch bowl fruit salad instructions in a quart punch bowl mix all of the cans of fruit juice and '], shape=(5,), dtype=string)
tf.Tensor(
[b'r||x|tkkqyff jacrj|ynkkoglychlpydveqwbysqzozpxj?ivsvatbyyyr?w?cceil pbif?yhxemwn jzoxy?vjqjoycfag?geoxzrgotuhfqaxklsyhyvhsmjfa'
 b'ra|cssodqh|l|hpyyfe|?zdlglriqzhydgjqwnsbziqpfe?fp yysix y|ash vylhzpbvfibdcefe  gz|zwafsqvzciqyfbweoxvvocli ww?orynghyuhsmv|a'
 b'wasxlzole|zptpycyz?yl?xbu|hdhnvwoktq w|sbxtivo tni|se?s?gyhrudybch

In [18]:

trained_chef = train_and_eval(trained_chef, imitation_ds_training, imitation_ds_exam, config)

tf.Tensor(
[b'ctions prepare peppers remove white sections mix ground beef rice egg and seasoning fill peppers place in deep cooking v'
 b'htly salted boiling water cook red cabbage for  minutes then draintitle german potato salad instructions peel and cut pot'
 b'gs bring to a boil and then reduce heat cover and simmer for  to  hours during the last hour of cooking add the winetitl'
 b'soup sour cream and cheese stir well add potatoes to mixture and pour into a  quart casserole dish crush cornflakes mi'
 b'yield   dozentitle punch bowl fruit salad instructions in a quart punch bowl mix all of the cans of fruit juice and '], shape=(5,), dtype=string)
tf.Tensor(
[b'te ceger blousinntle ir cnutheese and  o fook  add min oftin pe wit  tper and  are n  ofrf st r  r feet  te chon wut al  '
 b'th soup int on p mantete inttrectiens br ted tof aed bake sol bitler ce tilatren ton tep oruatinles inddr and getar morofgee '
 b' rlen and aor iatins iled bd ce srind cn cren tine in t plewive boitist

In [17]:
jnp.asarray(x)

Array([[14,  9,  9, ..., 13,  5,  5],
       [17,  3, 10, ...,  3,  5,  2],
       [ 2, 10,  3, ...,  3, 13,  2],
       ...,
       [ 2, 11,  4, ..., 22,  6,  7],
       [ 5,  2, 21, ...,  3,  2,  6],
       [ 3, 17,  8, ...,  7,  2, 10]], dtype=int32)

In [13]:
out = chef.apply_fn({"params": chef.params}, jnp.asarray(x), rngs={"dropout": jax.random.PRNGKey(0)}, training=False)

recipe_reader.decode(x, itos)[:32], recipe_reader.decode(jnp.argmax(jnp.squeeze(out), axis=-1), itos)[:5]


(<tf.Tensor: shape=(32,), dtype=string, numpy=
 array([b'ctions prepare peppers remove white sections mix ground beef rice egg and seasoning fill peppers place in deep cooking v',
        b'htly salted boiling water cook red cabbage for  minutes then draintitle german potato salad instructions peel and cut pot',
        b'gs bring to a boil and then reduce heat cover and simmer for  to  hours during the last hour of cooking add the winetitl',
        b'soup sour cream and cheese stir well add potatoes to mixture and pour into a  quart casserole dish crush cornflakes mi',
        b'yield   dozentitle punch bowl fruit salad instructions in a quart punch bowl mix all of the cans of fruit juice and ',
        b' often last fold in chocolate chip mints bake at  for  to  minutes or until done cake mix directionstitle cuddy f',
        b'ne at a time beating after each egg until mixture has satinlike sheen use pastry bag with large star tip and fill with mixtu',
        b'ions heat oven to  p

In [97]:
dropout_rng = jax.random.PRNGKey(0)
for batch_idx, (x, y) in enumerate(imitation_ds_training.take(1).as_numpy_iterator()):
    y = jax.nn.one_hot(y, len(itos.get_vocabulary()))
    loss_eval, logits_eval = eval_step_jitted(chef, x, y, dropout_rng=dropout_rng)

In [98]:
recipe_reader.decode(jnp.argmax(jnp.squeeze(logits_eval), axis=-1), itos)

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'tggdehtmuucnuglujcvsinsftlmaasaxmyirbqvyefgir?yfjkqa nxnbndgvmyxwejgokyam j hhmtcb dhppe svm|xdtilyystl?wqcfndjhhk|lungb?inhbvhu',
       b'tlgdehtmuucpugljgvsi|kftbmvasaxmyirbqvy|fg|r?yfjkuciaxwbwdgbayehejgrklgd j h mncb nhpreks?muqdailyysth?wqctncjhmn|ltkgb?inhbmhu',
       b'tggduhtmxucnulljcvsynkftlmaysaxuyi?bqvyafjaroyfjkqaaxwbpdrbmyewx gokcqm v w mnc| dhpueps|muqdpilymstl?tqctncjelqmlungb?rnhbvhu',
       b'tlgdehtmuucpkglujcvsinmftldvtsaxjirb?vy|fyir?yvjkqaaxwbndrdmyoiesgrkcum j h mtcb hhprpks?muxsailyystl?xqcfnvjhlm|dukgb?inhbvhu',
       b'clgdeh|muucpmgl cvsinmft?vvtsammjirbqhbegyipyy?ciqc axwbrdndayxhxjgikcum x hmtcb lvppi??m|x?ahltysalewq|f cjhlm|dukqbxichbvmu',
       b'tlgduhtaxucpuglujcvsy|pftbmvlsxxvjirb?vyafjarayfjkqa axwbndydmyqwejgrklqm ? hhmtcb hhpfeks?muqdtilyystl?eqcf cjhjq|gungb?rnhbmhu',
       b'tlgdehttuumdkgld mvs||pftbdvasammyirb?vyvggid?yfjkuaiaxwbwdydgyvwejgrkygx j hmtcb hhpueks?euxddilaysthojqgkfcjhmm

In [67]:
recipe_reader.decode(jnp.argmax(jnp.squeeze(logits_eval), axis=-1), itos)

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'gviegmkxlrjozhhlfs?ouhdmgtmufbrvstw|lhbcallehmdstsvclwykpyodufsnpn|px?crad ?od zmncashzizwwln?wfzcywnuwyricboqiijmwqgtzdxie  ',
       b'gvieernjf?jozhhlbs?oujmgbnufurcstytlrzcauuehmdst?vpuuyxpzdglfksnpxfqr?hradh?oicamncnjtzizqwln|wfzfyrfsdfaiubvdiijmjqcfqv?caxn',
       b'gviuvicrzjozhzgbj?oujagtm?fujpszktlrb eluehmdsivvdluykyzofufiszpmtpr?jdad??ogmagkcnjyzizexlw?wizc|wnswya?edqhikgvqgtqvxcex ',
       b'kviurxbxlzjozhhjfm?ouhdmglm?fzavol|lrba l?hmdstsydluyk|yocufisnlntoq?crlj??og  mncshzikwolw?izdywnuwyk?ercqpijtvogfuxxbexn',
       b'zvieurxnxlzjozhhfm?ouhmgbm?fbacslydhbeaflehmdstsvdlzkpzofufisn?nmpx?c ad?od zmncstzi?doln?wfkcywnsgykivbqpijmwagfuvxiezn',
       b'zvieurkxr?jozhh fm?otumgbmubzcsty|lrzaxl|hmdstvvdlukrzodufisnptmpx?hd???odwzmxckwtzikdwln?izfywfsdrkivdnqpifmwqgbqvyiazn',
       b'zvieurifczjozhhlbm?ouumgbmvqnacsty|drzcaxu|hmdst?vcucyopzdfctksmlxcpxrhad ?odzzmnckjtzikqwln?wizfyrfsdrkidfqiijmwqcfbvui xn',
       b'gvi

In [15]:



prng = jax.random.PRNGKey(0)
prng, dropout_rng = jax.random.split(prng)
chef = prepare_chef_for_training(config)

In [18]:

for epoch in range(200):
    ckpt = {'model': chef}
    for batch_idx, (x, y) in enumerate(imitation_ds_training.take(200).as_numpy_iterator()):
        y = jax.nn.one_hot(y, len(itos.get_vocabulary()))
        chef, (loss, logits, grads) = train_step_jitted(chef, x, y, dropout_rng=dropout_rng)
        if batch_idx % 100 == 0:
            print(f"epoch: {epoch}, batch: {batch_idx}, loss: {loss}")


epoch: 0, batch: 0, loss: 3.418436050415039


Exception ignored in: <function _xla_gc_callback at 0x17fac2700>
Traceback (most recent call last):
  File "/Users/stefruinard/Documents/personal/projects/202309_symbiotic_transformers/symbiotic_transformers/venv/lib/python3.9/site-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x17fac2700>
Traceback (most recent call last):
  File "/Users/stefruinard/Documents/personal/projects/202309_symbiotic_transformers/symbiotic_transformers/venv/lib/python3.9/site-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


epoch: 0, batch: 100, loss: 2.972416400909424


Exception ignored in: <function _xla_gc_callback at 0x17fac2700>
Traceback (most recent call last):
  File "/Users/stefruinard/Documents/personal/projects/202309_symbiotic_transformers/symbiotic_transformers/venv/lib/python3.9/site-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


epoch: 1, batch: 0, loss: 2.957674980163574



KeyboardInterrupt



In [ ]:

for batch_idx, (x, y) in enumerate(imitation_ds_training.take(1).as_numpy_iterator()):
    y = jax.nn.one_hot(y, len(itos.get_vocabulary()))
    chef, (loss, logits, grads) = train_step_jitted(chef, x, y, dropout_rng=dropout_rng)

In [19]:
jax.devices()

[CpuDevice(id=0)]

In [19]:
import os
os.listdir('/tmp/flax_ckpt/orbax/managed')

['31.orbax-checkpoint-tmp-1696064671479759', '29', '30']

In [23]:
ls_step = checkpoint_manager.latest_step()
restored_chef = checkpoint_manager.restore(ls_step)



In [29]:
restored_chef["model"]

{'opt_state': [{'count': array(0, dtype=int32),
   'mu': {'IdeaArticulation_0': {'Dense_0': {'bias': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0

In [37]:
reloaded_logits = chef.apply_fn({"params": restored_chef["model"]["params"]}, x, rngs={"dropout": dropout_rng})
recipe_reader.decode(jnp.argmax(jnp.squeeze(reloaded_logits), axis=-1), itos)

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'cy cooeqpzjzuaaqejvejwlwdd w| igf?poysxwypcgmat ?pc|rqccvmcssmkfryggdeltkfuzbnit?zjvfr||rsmke zbnk|kef?yrkfjibslfatsbhhoc jn',
       b'fjvxeqpzwlkprjmjscjaaskq wrzqgevloymxqxycqoatebh|jsshxvopgrxtifstghttdznkfmdtn?? znvuxm|smzinajbnk|qfq?b kfjsfrqowjcumhazcmln',
       b'dyhdteq mwvbafsvpejakisqcx|xhwkr?oywycpatmateupn|rshynmxervirfs|gmkdrlyrumbtnxmhzuafxkeisnrskhbnkbkfw?c ?txiastnatrwhfyn?jn',
       b'd vvkoeqnzd?illge?vojadpsvy qcixq pyawycxytqdq p|zehjly?erlcxzivglqtlytlqmulmi??ziaenm|mhnzboh|knkl|?tkhakfbdgym|faxsdhrvn|bn',
       b'cvdboeypzjvilrqrppoj|vpgdvw|cice paybywx|nqlqr?ptvqhmkrskmtkrpwvzpndtunrfdrbmitecxz?fkqpgziuthulu|keqkhrpfbitsivfycsxhac|ln',
       b'ctvvzeqlzntiprgvpbjakpdvxx|ei?fvjyyoxwxvaqkfvyipq|sshiy|?ermiske gmujrpyrfudtn??r fkmsksmr|utiunnlb?frdrtvjsaxipfaxsdhac?lm',
       b'chfyoyqppbzkhrg psejta?waiu| hzhohowwxwsylqgatr qc|rshtvzrvb??jfbkkzbbnrj zbni?acxv hn|rylziqhiuqklkesmdrfsbshxlmfimqhhajcpl',
   

In [ ]:

new_chef = orbax_checkpointer.restore('/tmp/flax_ckpt/orbax/managed')
for batch_idx, (x, y) in enumerate(imitation_ds.as_numpy_iterator()):
    y = jax.nn.one_hot(y, len(itos.get_vocabulary()))
    new_chef, (loss, logits) = train_step_jitted(new_chef, x, y, dropout_rng=dropout_rng)

In [190]:
logits.shape, y.shape


((32, 128, 30), (32, 128, 30))

In [74]:
rng = jax.random.PRNGKey(0)
for i in range(10000):
    rng, dropout_rng = jax.random.split(rng)
    chef, (loss, logits) = train_step_jitted(chef, x, y, dropout_rng=dropout_rng)
    if i % 50 == 0:
        print(loss)
        print(jnp.argmax(jnp.squeeze(logits), axis=-1), jnp.argmax(jnp.squeeze(y), axis=-1))


3.4371824
[[19 24  2 ...  3  2  9]
 [19 24  6 ...  3 26 29]
 [23 24  6 ...  3 26  6]
 ...
 [27 14  6 ...  5 26  2]
 [23 24  6 ...  2 27  7]
 [23 24 19 ...  8 26  2]] [[ 2 18  6 ... 11 23  6]
 [18  6 12 ... 11  2 10]
 [ 2 12  6 ...  9 10  7]
 ...
 [11  8  7 ... 14 22  3]
 [ 4 16  2 ... 20 15  4]
 [ 2 20  3 ... 13  6  3]]
2.8484073
[[15 12  6 ... 14  5  2]
 [ 3 15 18 ... 14  5  2]
 [11 12  6 ... 11  5 10]
 ...
 [12  7  6 ...  6  3  2]
 [ 4  7 18 ... 14  3  1]
 [ 3 12  2 ...  6  7  4]] [[ 2 18  6 ... 11 23  6]
 [18  6 12 ... 11  2 10]
 [ 2 12  6 ...  9 10  7]
 ...
 [11  8  7 ... 14 22  3]
 [ 4 16  2 ... 20 15  4]
 [ 2 20  3 ... 13  6  3]]
1.8854077
[[ 5 10  8 ... 14 10  5]
 [18  5  2 ... 11  6 10]
 [ 5 12  6 ... 10 10  7]
 ...
 [ 2  7 10 ... 14  4  4]
 [ 4 16  2 ...  4 15  2]
 [ 2 20  2 ...  4  6  3]] [[ 2 18  6 ... 11 23  6]
 [18  6 12 ... 11  2 10]
 [ 2 12  6 ...  9 10  7]
 ...
 [11  8  7 ... 14 22  3]
 [ 4 16  2 ... 20 15  4]
 [ 2 20  3 ... 13  6  3]]
0.8798094
[[11 18  6 ... 11 23  6]

KeyboardInterrupt: 

In [197]:
print(jnp.argmax(jnp.squeeze(logits), axis=-1)[:5] == jnp.argmax(jnp.squeeze(y), axis=-1)[:5])

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True

In [192]:
recipe_reader.decode(x, itos)[0]


<tf.Tensor: shape=(), dtype=string, numpy=b'ok slowly stirring occasionally  minutes or until meat is tender serve over rice or buttered noodles yields  to  servings'>

In [39]:
recipe_reader.decode(jnp.argmax(jnp.squeeze(logits), axis=-1), itos)

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'ptee and depfel and feape pn  bottteedsp inn basa sot with ranoining  totlp ons of punge  bake at  for  houeitle n',
       b'r cleaniand glice berrie wirh aid sric  atplesade ve fperigg on appr s doolappras stice munanau wasp and cot gripes in h',
       b'n sruterp rroed and cat wth bi chnt ou trr  cirotl mross oels  ith perted org ang soed redy be ce call get miorir aree  ',
       b'atr ctaens cieed witatb bbooe and pay untar onrn shace aried shil en pankstin ti c ravl boll stir in botrecoe sooceyamsecina ',
       b' ponstof  attar frkendt  for  hourtitle ltak dapeplar sterk instructionss roel sfore strurs insblfar crud  indtoelran',
       b'hernitel butte firt aiof mextorl oo booll tin  and nolar mix welr ade rhabeob in r an bea soucei o sea e   ro inch pana',
       b'utle coovee   ivg f to tsppek hip bagun ex uf ooy intt phociwate chip ald shrasfon tox are to syeenthec g dnn al fndkeyesgv',
       b'tibnsr mnx ingrepients 

In [141]:
recipe_reader.decode(
jnp.argmax(jnp.squeeze(logits), axis=-1).reshape(-1, 1),
itos
)

recipe_reader.decode(jnp.argmax(y, axis=-1), itos)

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'y', b' ', b' ', b't', b't', b'i', b'd', b'o', b'e', b'm', b'i',
       b'l', b'e', b'h', b'n', b's', b't', b'l', b'h', b's', b'i', b' ',
       b'n', b'f', b' ', b'd', b's', b'o', b'f', b'r', b'h', b'o'],
      dtype=object)>

In [79]:
jnp.argmax(jnp.squeeze(logits), axis=-1), y[:, -1]

(Array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 2, 2, 2, 2], dtype=int32),
 array([10, 20, 10,  9,  9,  2,  5, 17,  3,  7, 11,  8,  4, 22, 10, 12, 10,
        20, 19, 26,  2,  2, 10,  6,  6, 12,  3, 18, 22,  7,  6,  3]))

Array([[[-2.6592684 , -1.6256614 , -1.4934324 , ..., -1.4009198 ,
         -1.1144856 , -1.5977932 ]],

       [[-2.0052314 , -0.45797104, -0.5283097 , ..., -2.510156  ,
         -0.22983934, -0.13950777]],

       [[-0.57558966, -1.0191025 , -2.2016664 , ..., -3.0531387 ,
         -0.3830731 , -1.412798  ]],

       ...,

       [[-0.57177866, -0.13992631, -2.1057372 , ..., -2.6731741 ,
         -1.4790828 , -3.0105035 ]],

       [[-1.2264494 , -1.0429674 , -0.26391634, ..., -3.3369868 ,
         -0.5846946 , -2.311417  ]],

       [[-2.279438  ,  0.06160758, -1.3319504 , ..., -2.0458322 ,
         -0.41904426, -1.8828329 ]]], dtype=float32)